# 03 Challenges

## Extracting rows

One thing we always want to do is extract rows based on some criteria.
For example, in Reaction Time data we often trim the early responses as
anticipatory, and the late responses as outliers.

**Challenge 1**: Find and remove the rows with NAs, save it out to
"newdf"


In [8]:

newdf <- ...

head(newdf)

ERROR: Error in eval(expr, envir, enclos): '...' used in an incorrect context


**Challenge 2**: Filter out RT's below 150 milliseconds and show how
many you've filtered out.



In [ ]:

newdf %>%
  filter(...) %>%
  summarise(...)




**Challenge 3**: Get the number of observations (rows) kept and removed
with <code>group_by()</code>


In [ ]:


newdf %>%
  group_by(...) %>%
  summarise(...)



In papers, you want to report the proportion of data you have removed
and the criteria with which you removed that data.

Now you have the slow responses, but a more appropriate standard might
be removing data above three standard deviations from the mean for each
participant's own distribution. Let's try that now...


In [ ]:

newdf %>% #Step 1 is to group by the subject and get the mean and sd
  group_by(s) %>%
  mutate(mRT = mean(RT), RTsd = sd(RT))

newdf %>% #Step 2 is to make a cutoff 3 standard deviations above the mean for each participant
  group_by(s) %>%
  mutate(mRT = mean(RT), RTsd = sd(RT)) %>%
  mutate(cutoff = mRT + 3*RTsd)


newdf %>% #Step 3 Let's clean up the columns we need and the ones we don't
  group_by(s) %>%
  mutate(mRT = mean(RT), RTsd = sd(RT)) %>%
  mutate(cutoff = mRT + 3*RTsd) %>%
  select(-mRT, -RTsd)


newdf %>% #Step 4 Now let's filter those RTs above the cutoff and count them
  group_by(s) %>%
  mutate(mRT = mean(RT), RTsd = sd(RT)) %>%
  mutate(cutoff = mRT + 3*RTsd) %>%
  select(-mRT, -RTsd) %>%
  filter(RT > cutoff) %>%
  summarise(n = n()) %>% #spread it out so its easy to read
  pivot_wider(names_from = s, values_from = n)


newdf %>% #We can see the proportion of removed data
  group_by(s) %>%
  mutate(mRT = mean(RT), RTsd = sd(RT)) %>%
  mutate(cutoff = mRT + 3*RTsd) %>%
  select(-mRT, -RTsd) %>%
  group_by(RT > cutoff | RT < 150) %>%
  summarise(n = n()) %>%
  mutate(prop = (n/sum(n))*100)

dfclean <- newdf %>% #Save it out as dfclean
  group_by(s) %>%
  mutate(mRT = mean(RT), RTsd = sd(RT)) %>%
  mutate(cutoff = mRT + 3*RTsd) %>%
  select(-mRT, -RTsd) %>%
  filter(RT < cutoff & RT > 150) #The RTs must be less than the cutoff AND greater than 150 ms



Now we're ready to do some descriptive statistics with the groups

**Challenge 4**: What is the mean reaction time and standard deviation
for congruent vs incongruent trials



In [ ]:

#hint group_by(the factor you're interested in)
dfclean %>%
  group_by(...) %>%
  summarise(...)



**Challenge 5**: Get the mean and sd of participant means for congruent
vs incongruent trials and compare to the previous outcome.



In [ ]:

dfclean %>%
  group_by(...) %>%
  summarise(...) %>%
  group_by(...) %>%
  summarise(..., ...)


This is a quirk of repeated measures fully within designs, as the levels
of analysis are nested. More on that later. We can quickly visualize
some of these things, like the distribution of responses in base R.
Note: Soon we will learn to visualize everything in ggplot2.


In [ ]:

hist(dfclean$RT, breaks = 100)

#Add a line for the mean
hist(dfclean$RT, breaks = 100)
abline(v = mean(dfclean$RT),
       col = "red",
       lwd = 2)
#add a line for the median
abline(v = median(dfclean$RT),
       col = "blue",
       lwd = 3)
lines(density(dfclean$RT),
      lwd = 2, # thickness of line
      col = "chocolate3")# density plot

#Add a density plot,
hist(dfclean$RT, breaks = 100,
     prob = T)
lines(density(dfclean$RT),
      lwd = 2, # thickness of line
      col = "chocolate3")# density plot
abline(v = mean(dfclean$RT),
       col = "red",
       lwd = 2)
#add a line for the median
abline(v = median(dfclean$RT),
       col = "blue",
       lwd = 3)

legend(x = "topright", # location of legend within plot area
       c("Density plot", "Mean", "Median"),
       col = c("chocolate3", "red", "blue"),
       lwd = c(2, 2, 3))





## Pivoting, Separating and Uniting

look at this <code>tibble(data.frame)</code>


In [ ]:

stocks <- tibble(
  year   = c(2015, 2015, 2016, 2016),
  half  = c(   1,    2,     1,    2),
  return = c(1.88, 0.59, 0.92, 0.17)
)



<code>pivot_longer</code> and
<code>pivot_wider</code> are not opposites.





In [ ]:


stocks %>%
  pivot_wider(names_from = "year",values_from = "return") %>%
  pivot_longer(cols = `2015`:`2016`,names_to = "year")






You can make data.frames with space using the <code>tribble()</code>
function



In [ ]:
```{r tribble}
people <- tribble(
  ~name,             ~key,    ~value,
  #-----------------|--------|------
  "Phillip Woods",   "age",       45,
  "Phillip Woods",   "height",   186,
  "Jessica Cordero", "age",       37,
  "Jessica Cordero", "height",   156
)

people

```


**Challenge 6**: spread people so height and age have their own column



In [ ]:


newpeople <- people %>%
  pivot_wider(...)




**Challenge 7**: gather newpeople so that height and age are back together



In [ ]:


newpeople %>%
  pivot_longer(...)



You can separate or unite columns based upon a separator



In [ ]:

people %>%
  separate(name, into = c("first","last"))




How did it know? We usually have to specify the separator



In [ ]:

weirdpeople <- people %>%
  separate(name, into = c("befored","afterd"), sep = "d")




Notice that the separator disappears

**Challenge 8**: Unite the two columns above with "d" as a separator



In [ ]:

weirdpeople %>%
  unite(...)

